In [4]:
import requests
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

from bs4 import BeautifulSoup

parser = BeautifulSoup(website_url,'html.parser')
table = parser.find('table',{'class':'wikitable'})
#print(table.tr.text)
headers="Postcode,Borough,Neighbourhood"
table1=""
for tr in table.find_all('tr'):
    row1=""
    for tds in tr.find_all('td'):
        if not tds.text.strip():
            row1=row1+"Not assigned,"        
        else:
            newText = tds.text.strip().replace(",", " ")
            row1=row1+newText+","                
    row1=row1[:-1:]+"\n"
    table1=table1+row1

In [5]:
file=open("toronto.csv","wb")
file.write(bytes(headers,encoding="ascii",errors="ignore"))
file.write(bytes(table1,encoding="ascii",errors="ignore"))

7136

In [35]:
import pandas as pd
df = pd.read_csv('toronto.csv')
df.columns=["Postcode","Borough","Neighbourhood"]
df=df[df['Borough']!="Not assigned"]
df = df.groupby(['Postcode','Borough'], sort=False).agg( ', '.join)
df['Neighbourhood'] = df['Neighbourhood'].apply(lambda x: df['Borough'] if x == "Not assigned" else x)
df=df.reset_index()
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [36]:
df_geo = pd.read_csv('http://cocl.us/Geospatial_data')
df_geo.columns=['Postcode','Latitude','Longitude']
df_geo.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [38]:
df_merge = pd.merge(df,df_geo,on='Postcode')
df_merge.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494
